# Load Excel Files as Pandas DataFrames without Hidden Rows and Columns

## Why `pandas.read_excel()` does not work?

In [10]:
import pandas as pd

df = pd.read_excel("data/titanic.xlsx")

df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


Observe that by default, the `read_excel()` method reads in data from all rows and columns from the specified Excel file. In other words, it does not exclude the hidden rows and columns. Thus, using this `read_excel()` method is insufficient.

## Loading Excel file and worksheet using `openpyxl`

To open an existing Excel file using the `openpyxl` package, we use the `openpyxl.load_workbook()` method, specifying the name of the path where the Excel file is stored.

In [11]:
import openpyxl 
import string

# Open an Excel workbook
workbook = openpyxl.load_workbook("data/titanic.xlsx")

This creates a `Workbook` object which, according to the documentation, is the "top-level container for all document information". This object contains many attributes pertaining to the input file, including the `.sheetnames` attribute which returns the list of the names of all worksheets in the workbook.

In [12]:
# Create a list of names of all worksheets in `workbook`
sheet_names = workbook.sheetnames

# Create a `Worksheet` object 
worksheet = workbook[sheet_names[0]]

In our `titanic.xlsx` file, we only have one worksheet named "train", so we get the sheet name by taking the first element of the `sheet_names` list. Next, we create a `Worksheet` object from the `Workbook` object.

## Finding indices of hidden rows

Similarly, the `Worksheet` object contains attributes pertaining to the specified worksheet. To find indices of all hidden rows, we make use of the `.row_dimensions` attribute of the `Worksheet` object, like this:

In [13]:
# List of indices corresponding to all hidden rows
hidden_rows_idx = [
    row - 2
    for row, dimension in worksheet.row_dimensions.items() 
    if dimension.hidden
]

print(hidden_rows_idx)

[4, 9, 14, 19]


Notice that we need to take `row - 2` instead of just `row` because we are interested to find indices corresponding to the Pandas DataFrame, not the Excel file.

## Finding names of hidden columns

To find names of all hidden columns, we first use the `.column_dimension` attribute of the `Worksheet` object:

In [14]:
# List of indices corresponding to all hidden columns
hidden_cols = [
    col 
    for col, dimension in worksheet.column_dimensions.items() 
    if dimension.hidden
]

print(hidden_cols)

['F', 'I', 'K']


Notice that the `hidden_cols` list comprises uppercase alphabets corresponding to the hidden columns of an Excel worksheet. We need to convert the `hidden_cols` list into a list of names of the hidden columns. To do so, we use Python's built-in library, `string`, and its `.ascii_uppercase` attribute.

In [15]:
# List of indices corresponding to all hidden columns
hidden_cols_idx = [
    string.ascii_uppercase.index(col_name) 
    for col_name in hidden_cols
]

# Find names of columns corresponding to hidden column indices
hidden_cols_name = df.columns[hidden_cols_idx].tolist()

print(hidden_cols_name)

['Age', 'Ticket', 'Cabin']


**Disclaimer: Do note that using `string.ascii_uppercase` assumes that there are at most 26 columns. If there are more than 26 columns, the code will need to be modified.**

Finally, once we have the indices for the hidden rows and names for the hidden columns, the rest is simple. To exclude those hidden rows and columns, we simply use Pandas' `.drop()` method.

In [16]:
# Drop the hidden columns
df.drop(hidden_cols_name, axis=1, inplace=True)

# Drop the hidden rows
df.drop(hidden_rows_idx, axis=0, inplace=True)

# Reset the index
df.reset_index(drop=True, inplace=True)

df

,PassengerId,Survived,Pclass,Name,Sex,SibSp,Parch,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,1,0,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,1,0,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,0,0,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,1,0,53.1000,S
4,6,0,3,"Moran, Mr. James",male,0,0,8.4583,Q
5,7,0,1,"McCarthy, Mr. Timothy J",male,0,0,51.8625,S
6,8,0,3,"Palsson, Master. Gosta Leonard",male,3,1,21.0750,S
7,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,0,2,11.1333,S
8,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,1,1,16.7000,S
9,12,1,1,"Bonnell, Miss. Elizabeth",female,0,0,26.5500,S


## Putting It All Together

In [17]:
# Import modules
import openpyxl 
import pandas as pd
import string

# Read Excel file as Pandas DataFrame
df = pd.read_excel("data/titanic.xlsx")

# Open an Excel workbook
workbook = openpyxl.load_workbook("data/titanic.xlsx")

# Create a `Worksheet` object 
worksheet = workbook[workbook.sheetnames[0]]

# List of indices corresponding to all hidden rows
hidden_rows_idx = [
    row - 2
    for row, dimension in worksheet.row_dimensions.items() 
    if dimension.hidden
]

# List of indices corresponding to all hidden columns
hidden_cols_idx = [
    string.ascii_uppercase.index(col_name) 
    for col_name in [
        col 
        for col, dimension in worksheet.column_dimensions.items() 
        if dimension.hidden
    ] 
]


# Find names of columns corresponding to hidden column indices
hidden_cols_name = df.columns[hidden_cols_idx].tolist()

# Drop the hidden columns
df.drop(hidden_cols_name, axis=1, inplace=True)

# Drop the hidden rows
df.drop(hidden_rows_idx, axis=0, inplace=True)

# Reset the index
df.reset_index(drop=True, inplace=True)

df

,PassengerId,Survived,Pclass,Name,Sex,SibSp,Parch,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,1,0,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,1,0,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,0,0,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,1,0,53.1000,S
4,6,0,3,"Moran, Mr. James",male,0,0,8.4583,Q
5,7,0,1,"McCarthy, Mr. Timothy J",male,0,0,51.8625,S
6,8,0,3,"Palsson, Master. Gosta Leonard",male,3,1,21.0750,S
7,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,0,2,11.1333,S
8,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,1,1,16.7000,S
9,12,1,1,"Bonnell, Miss. Elizabeth",female,0,0,26.5500,S
